In [133]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


def __check_prediction(data, index) -> bool:
    return (data["Closing price"].iloc[index+ 1] > data["Closing price"].iloc[index]
            or data["Closing price"].iloc[index+ 2] > data["Closing price"].iloc[index]
            or data["Closing price"].iloc[index+ 3] > data["Closing price"].iloc[index]
            or data["Closing price"].iloc[index+ 4] > data["Closing price"].iloc[index]
            or data["Closing price"].iloc[index+ 5] > data["Closing price"].iloc[index])


def __print_info(data, index, prediction):
    print("Today: ", data["Closing price"].iloc[index],)
    print("Next days: ", '\n', 
            "T1", data["Closing price"].iloc[index+ 1], '\n',
            "T2", data["Closing price"].iloc[index+ 2], '\n',
            "T3", data["Closing price"].iloc[index+ 3], '\n',
            "T4", data["Closing price"].iloc[index+ 4], '\n',
            "T5", data["Closing price"].iloc[index+ 5])
    print("Prediction: ", prediction[0])
    print("-----")


def test(today, num_of_days, data, model, scaler, sequence_length) -> int:
    
    specific_date = today  # Adjust this date as needed

    # Extract the data for the specific date and the preceding SEQUENCE_LENGTH - 1 dates
    specific_date_index = data.index.get_loc(specific_date)

    num_of_correct_predictions = 0

    for x in range(num_of_days):

        input_features = data["Differenced Closing"].iloc[specific_date_index - sequence_length + 1 : specific_date_index + 1].values
        
        # Reshape the input features to match the input shape expected by the LSTM model
        input_features_reshaped = input_features.reshape(1, sequence_length, 1)  # Assuming one feature dimension

        input_features_scaled = scaler.transform(input_features_reshaped.reshape(-1, 1))
        input_features_scaled = input_features_scaled.reshape(1, sequence_length, 1)

        # Make predictions using the model
        prediction = model.predict(input_features_scaled)

        if __check_prediction(data, specific_date_index):
            num_of_correct_predictions += 1

        __print_info(data, specific_date_index, prediction)
        
        specific_date_index += 1

    return num_of_correct_predictions



In [134]:
import pandas as pd
from tensorflow.keras.models import load_model
import joblib

# data = pd.read_csv("SHB_A-1987-01-02-2023-09-08.csv", sep=";", decimal=",", skiprows=1, encoding="utf-8")

# data.columns
column_order = ['Date', 'Bid', 'Ask', 'Opening price', 'High price', 'Low price', 'Closing price', 'Average price', 'Total volume', 'Turnover', 'Trades']
data = pd.read_csv("../../Data/Raw_data/SHB_A-1999-01-06-2024-02-05.csv", sep=";", decimal=",", skiprows=1, usecols=column_order)

pd.options.mode.chained_assignment = None  # default='warn'

columns_to_drop = ["Bid", "Ask", "Opening price", "Average price", "Trades"]
data = data.set_index("Date").drop(columns=columns_to_drop)

data = data.dropna(axis=1)

data = data[::-1]

data["Differenced Closing"] = data["Closing price"].diff()


data

,Closing price,Differenced Closing
Date,,
1999-01-07,39.111,NaN
1999-01-08,38.889,-0.222
1999-01-11,37.778,-1.111
1999-01-12,36.944,-0.834
1999-01-13,35.611,-1.333
...,...,...
2024-01-29,113.300,-1.100
2024-01-30,113.650,0.350
2024-01-31,112.550,-1.100


In [135]:
model = load_model("/Users/erikolofsson/Documents/Kodning/Python/StockPricePredictor/App/Scripts/Models/my_model.keras")
scaler = joblib.load("/Users/erikolofsson/Documents/Kodning/Python/StockPricePredictor/App/Scripts/Models/my_scaler.pkl")


num_of_runs = 5

correct_predictions = test("2024-01-10", num_of_runs, data, model, scaler, 5)

print("ACC: ", correct_predictions / num_of_runs)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 7 variables whereas the saved optimizer has 12 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
Today:  109.2
Next days:  
 T1 109.55 
 T2 110.65 
 T3 110.4 
 T4 107.25 
 T5 107.55
Prediction:  [0.9982113]
-----
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Today:  109.55
Next days:  
 T1 110.65 
 T2 110.4 
 T3 107.25 
 T4 107.55 
 T5 107.9
Prediction:  [0.99858993]
-----
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Today:  110.65
Next days:  
 T1 110.4 
 T2 107.25 
 T3 107.55 
 T4 107.9 
 T5 107.85
Prediction:  [0.99878323]
-----
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Today:  110.4
Next days:  
 T1 107.25 
 T2 107.55 
 T3 107.9 
 T4 107.85 
 T5 110.1
Prediction:  [0.99904025]
-----
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Today:  107.25
Next days:  
 T1 107.55 
 T2 107.9 
 T3 107.85 
 T4 110.1 
 T5 109.65
Prediction:  [0.9986723]
-----
ACC:  0.6
